## Doing quantitative stuff as a quantitative Quant the quantitative way
### Thomas Schmelzer

+ #### Academia (2004-2007) 
    + PhD Mathematics, Oxford University


+ #### 7 Years in Hedge Funds (2007-2014) 
    + Winton Capital (similar to Geneva's Systematica)
    + IMC Trading (formerly known as the pride of Switzerland's prop trading)
    + OxAM Liechtenstein


+ #### since 2014
    + Head of Quant Research for Lobnek Wealth Management (Geneva)

# Linear Regression

Let $\mathbf{A} \in \mathbb{R}^{n \times m}$ and $\mathbf{b} \in \mathbb{R}^n$. Solve the unconstrained least squares problem: 

\begin{align}
\mathbf{x}^{*}=\arg\min_{\mathbf{x} \in \mathbb{R}^m}& \rVert{\mathbf{A}\mathbf{x}-\mathbf{b}}\lVert_2
\end{align}

The $i$th column of $\mathbf{A}$ may represent the time series of returns for asset $i$. Portfolio Optimisation is about combining assets in a clever way. 



## The normal equations

As we (probably) all know 

$$
\mathbf{x}^{*}=\left(\mathbf{A}^T \mathbf{A}\right)^{-1}\mathbf{A}^{T}\mathbf{x}
$$
solves
\begin{align}\mathbf{x}^{*}=\arg\min_{\mathbf{x} \in \mathbb{R}^m}& \rVert{\mathbf{A}\mathbf{x}-\mathbf{b}}\lVert_2
\end{align}

But...


## Constrained regression

Let $\mathbf{A} \in \mathbb{R}^{n \times m}$ and $\mathbf{b} \in \mathbb{R}^n$. We solve the constrained least squares problem: 

\begin{align}\mathbf{x}^{*}=\arg\min_{\mathbf{x} \in \mathbb{R}^m}& \rVert{\mathbf{A}\mathbf{x}-\mathbf{b}}\lVert_2\\
\text{s.t. } &\Sigma\,x_i=1\\
            &\mathbf{x}\geq 0
\end{align}

## Thales of Miletus (c. 624 BC -  c. 546 BC). 
<img src="talk/thales.jpg" style="margin-left:auto; margin-right:auto; display:block">